# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Данные геологоразведки:
1. id — уникальный идентификатор скважины;
2. f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
3. product — объём запасов в скважине (тыс. баррелей).

In [35]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [36]:
geo_data_reg_0 = pd.read_csv('/datasets/geo_data_0.csv')
display(geo_data_reg_0)
geo_data_reg_0.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [37]:
geo_data_reg_1 = pd.read_csv('/datasets/geo_data_1.csv')
display(geo_data_reg_1)
geo_data_reg_1.info()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [38]:
geo_data_reg_2 = pd.read_csv('/datasets/geo_data_2.csv')
display(geo_data_reg_2)
geo_data_reg_2.info()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


## Обучение и проверка модели

### Функция моделирования

In [39]:
def modeling(features_train, features_valid, target_train, target_valid):
    model = LinearRegression() # инициализация модели "линейная регрессия"
    model.fit(features_train, target_train) # обучение модели на обучающей выборке
    predictions = model.predict(features_valid) # предсказания модели на валидационной выборке
    rmse = mean_squared_error(target_valid, predictions) ** 0.5 # значение метрики RMSE на валидационной выборке
    
    return predictions, rmse, target_valid

**Важно**

Признак `id` для данного исследования интереса не представляют, т.к. являются служебной информацией, изночально уникальной для каждой скважины, поэтому этот столбец исключим из данных для создания и тестирования модели.

### Регион 0

In [40]:
#Выделение Признаков и Целевого признака
features = geo_data_reg_0.drop(['id','product',], axis=1)
target = geo_data_reg_0['product']

In [41]:
#Разбивка выборки на обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12091977)
target_valid

86601     42.679586
89058    104.522378
94735     60.898494
57141    108.557772
78255     45.963185
            ...    
3575     108.439854
29957     82.017013
53584    109.635221
73150     52.503049
91204     97.957026
Name: product, Length: 25000, dtype: float64

In [42]:
reg_0 = modeling(features_train, features_valid, target_train, target_valid)
print("RMSE модели линейной регрессии на валидационной выборке:", reg_0[1])
print("Cредний запас предсказанного сырья:", reg_0[0].mean())


RMSE модели линейной регрессии на валидационной выборке: 37.34738707044069
Cредний запас предсказанного сырья: 92.28299910080804


### Регион 1

In [43]:
#Выделение Признаков и Целевого признака
features = geo_data_reg_1.drop(['id','product',], axis=1)
target = geo_data_reg_1['product']

In [44]:
#Разбивка выборки на обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12091977)
features_valid

,f0,f1,f2
86601,0.777234,1.292963,3.005624
89058,10.615361,-10.037798,3.001227
94735,-4.433437,-7.679420,1.001997
57141,-5.648996,-14.101062,2.004878
78255,7.131766,-4.558284,2.008987
...,...,...,...
3575,-7.020147,-15.276959,5.001826
29957,-1.085298,1.183474,4.996948
53584,-12.375617,0.413205,-0.000935
73150,2.556158,0.808446,3.004397


In [45]:
reg_1 = modeling(features_train, features_valid, target_train, target_valid)
print("RMSE модели линейной регрессии на валидационной выборке:", reg_1[1])
print("Cредний запас предсказанного сырья:", reg_1[0].mean())


RMSE модели линейной регрессии на валидационной выборке: 0.8922847864260562
Cредний запас предсказанного сырья: 68.87605889990549


### Регион 2

In [46]:
#Выделение Признаков и Целевого признака
features = geo_data_reg_2.drop(['id','product',], axis=1)
target = geo_data_reg_2['product']

In [47]:
#Разбивка выборки на обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12091977)
features_valid

,f0,f1,f2
86601,-0.538323,-1.278551,-5.529411
89058,-2.314127,2.386062,4.231771
94735,-2.446468,3.195713,6.372292
57141,0.148190,0.939364,8.191370
78255,0.056715,-0.870233,4.628166
...,...,...,...
3575,-1.459180,0.598307,-1.561765
29957,-1.898419,-3.566105,2.737400
53584,3.665042,-0.856673,2.426073
73150,1.685306,-1.608017,5.121823


In [48]:
reg_2 = modeling(features_train, features_valid, target_train, target_valid)
print("RMSE модели линейной регрессии на валидационной выборке:", reg_2[1])
print("Cредний запас предсказанного сырья:", reg_2[0].mean())


RMSE модели линейной регрессии на валидационной выборке: 40.10506382882196
Cредний запас предсказанного сырья: 94.85133392288175


### Выводы по результатам исследования

Регион 0:
1. RMSE модели линейной регрессии на валидационной выборке: 37.34738707044069
2. Cредний запас предсказанного сырья: 92.28299910080804

Регион 1:
1. RMSE модели линейной регрессии на валидационной выборке: 0.8922847864260552
2. Cредний запас предсказанного сырья: 68.87605889990549

Регион 2:
1. RMSE модели линейной регрессии на валидационной выборке: 40.105063828821955
2. Cредний запас предсказанного сырья: 94.85133392288175

Распределение запасов по площади месторождения Региона 1, самое равномерное. Что это даёт? В начале разработки месторождения, можно строить скважины на определённом, локальном участке, т.е. избежать случая, когда надо строить скважины на диаметрально противоположных участках месторождения, для обеспечения требуемого объёма добычи. Локализация объектов строительства существенно сокращает расходы на инфраструктуру (дороги, линии электропередачи, объекты подготовки и транспортировки нефти, объекты быта и т.д.).

## Подготовка к расчёту прибыли

In [49]:
RESEARCH_POINT = 500 #количество исследованных точек при разведке региона
INVESTMENTS = 10**10 # Бюджет на разработку скважин в регионе, рублей
WELLS_COUNT = 200 # количество скважин для разработки
BARREL_PRICE = 450 # Цена за баррель нефти, рублей
well_product = INVESTMENTS / (WELLS_COUNT * BARREL_PRICE * 1000) # необходимый для окупаемости объём 
                                                                     #извлекаемых запасов из скважины (тыс. баррелей) 


In [50]:
well_product  

111.11111111111111

**Вывод:** полученный объём на одну скважину, для безубыточного производства, больше чем средний запас на одну скважину в каждом регионе. Что приводит нас к дополнительным расчётам, т.е. выбрать любую скважину из рассчитанных и начать её строить неразумно, необходимо найти скважины у которых расчётные запасы больше полученного объёма безубыточности и их уже строить и эксплуатировать.


### Функция расчёта прибыли

In [51]:
def revenue(target, predictions, wells_count):
    predict_sorted = predictions.sort_values(ascending=False)
    selected = target[predict_sorted.index][:wells_count]
    return 1000 * BARREL_PRICE * selected.sum() - INVESTMENTS


## Расчёт прибыли и рисков 

### Регион 0

In [52]:
%%time
state = np.random.RandomState(12345)
predictions_0 = pd.Series(reg_0[0], index=features_valid.index)

values = []
for i in range(1000):
    target_subsample = reg_0[2].sample(n=RESEARCH_POINT, replace=True, random_state=state)
    predic_subsample = predictions_0[target_subsample.index]
    
    values.append(revenue(target_subsample, predic_subsample, WELLS_COUNT))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()

print("Средняя прибыль:", mean)
print("2,5%-квантиль:", lower)
print("97,5%-квантиль:", upper)


Средняя прибыль: 441838876.1741194
2,5%-квантиль: -116066293.46359591
97,5%-квантиль: 1003495497.1075948
CPU times: user 1.31 s, sys: 0 ns, total: 1.31 s
Wall time: 1.33 s


In [53]:

risc_of_loss = values[values < 0].count() / len(values)
print("Вероятность убытков:",risc_of_loss * 100, "%")


Вероятность убытков: 5.0 %


### Регион 1

In [54]:
%%time
state = np.random.RandomState(12345)
predictions_1 = pd.Series(reg_1[0], index=features_valid.index)

values = []
for i in range(1000):
    target_subsample = reg_1[2].sample(n=RESEARCH_POINT, replace=True, random_state=state)
    predic_subsample = predictions_1[target_subsample.index]
    
    values.append(revenue(target_subsample, predic_subsample, WELLS_COUNT))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()

print("Средняя прибыль:", mean)
print("2,5%-квантиль:", lower)
print("97,5%-квантиль:", upper)

Средняя прибыль: 483036350.1829771
2,5%-квантиль: 74524080.2121541
97,5%-квантиль: 875294062.1564543
CPU times: user 1.27 s, sys: 0 ns, total: 1.27 s
Wall time: 1.28 s


In [55]:
risc_of_loss = values[values < 0].count() / len(values)
print("Вероятность убытков:",risc_of_loss * 100, "%")

Вероятность убытков: 1.5 %


### Регион 2

In [56]:
%%time
state = np.random.RandomState(12345)
predictions_2 = pd.Series(reg_2[0], index=features_valid.index)

values = []
for i in range(1000):
    target_subsample = reg_2[2].sample(n=RESEARCH_POINT, replace=True, random_state=state)
    predic_subsample = predictions_2[target_subsample.index]
    
    values.append(revenue(target_subsample, predic_subsample, WELLS_COUNT))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()

print("Средняя прибыль:", mean)
print("2,5%-квантиль:", lower)
print("97,5%-квантиль:", upper)

Средняя прибыль: 405326700.25333875
2,5%-квантиль: -171528232.68842882
97,5%-квантиль: 995447126.1677542
CPU times: user 1.29 s, sys: 0 ns, total: 1.29 s
Wall time: 1.3 s


In [57]:
risc_of_loss = values[values < 0].count() / len(values)
print("Вероятность убытков:",risc_of_loss * 100, "%")

Вероятность убытков: 7.8 %


### Выводы по результатам исследования

И так, свяжем средние значения и квантили с экономическими показателями выручки и риска. Сначала определимся, что же такое квантиль и с чем его едят. 

Кванти́ль в математической статистике — значение, которое заданная случайная величина не превышает с фиксированной вероятностью. Если вероятность задана в процентах, то квантиль называется процентилем или перцентилем.
Например, фраза «90-й процентиль массы тела у новорожденных мальчиков составляет 4 кг»[1] означает, что 90 % мальчиков рождаются с весом, меньшим либо равным 4 кг, а 10 % мальчиков рождаются с весом, большим 4 кг.

Из этого следует, что с 95% вероятностью значение прибыли в рублях для регионов будет находиться в диапазоне:
1. Регион 0 - от -116 066 293 до 1 003 495 497, при средней прибыли по региону 441 838 876;
2. Регион 1 - от 74 524 080 до 875 294 062, при средней прибыли по региону 483 036 350;
3. Регион 2 - от -171 528 232 до 995 447 126, при средней прибыли по региону 405 326 700.

Вероятность убытков по регионам:
1. Регион 0 - вероятность убытков: 5.0 %;
2. Регион 1 - вероятность убытков: 1.5 %;
3. Регион 2 - вероятность убытков: 7.8 %.

Выбирать регион для разработки будем исходя из следующих условий:
1. Средния прибыль по региону имеет наибольшее значение;
2. Вероятность убытка должна быть меньше 2,5%.

Выдвинутым условиям соответсвует Регион 1, его и будем разрабатывать первым.



## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован